# Team Ratings

In [18]:
# Third party libraries
import pandas as pd
import numpy as np

# Local libraries
import Tools.json_utils as ju

FILENAME = "Data/data_2024.json"

In [ ]:
# Currently unused (save for Massey, Colley, Elo Ratings)
tournament = Tournament.Tournament(url=TOURNAMENT_URL,
                                   debug=True)

In [15]:
# Example dataset
score_df = pd.DataFrame({
    "Home": [],
    "Home_Score": [],
    "Away": [],
    "Away_Score": []
})
print(score_df)

       Home  Home_Score      Away  Away_Score
0    Kansas          78      Duke          75
1  Arkansas          70    Kansas          68
2      Duke          65  Arkansas          52


In [40]:

# Read Stats
teams_df = pd.read_json(FILENAME)

team = "Kansas"

team_df = teams_df[team]
non_ncaa_idxs = []


for i in range(len(team_df["Type"])):
    if team_df["Type"][i] != "NCAA":
        non_ncaa_idxs.append(i)

for i in non_ncaa_idxs:
    print(i, team_df["Type"][i])
    print(team_df["Tm"][i])
    print(team_df["Opp"][i])
    print(team_df["Site"][i])

    # Find which team is home/away (None = home, @ = away, N = neutral/assign home to winner?)

# for t in list(df.keys()):
#     win = df[t]["W"][-1]
#     loss = df[t]["L"][-1]
#     print(f"{t}: {win}-{loss}")

0 REG
99
56
None
1 REG
99
61
None
2 REG
89
84
N
3 REG
83
56
N
4 REG
59
73
N
5 REG
69
60
N
6 REG
71
63
None
7 REG
69
65
None
8 REG
88
69
None
9 REG
73
64
None
10 REG
75
71
@
11 REG
75
60
None
12 REG
86
67
N
13 REG
83
81
None
14 REG
60
65
@
15 REG
78
66
None
16 REG
90
66
@
17 REG
85
91
@
18 REG
74
69
None
19 REG
75
79
@
20 REG
83
54
None
21 REG
78
65
None
22 REG
70
75
@
23 REG
64
61
None
24 REG
50
79
@
25 REG
67
57
@
26 REG
86
67
None
27 REG
68
76
None
28 REG
74
82
@
29 REG
90
68
None
30 REG
46
76
@
31 CTOURN
52
72
N


In [ ]:
# Get unique teams and index them
teams = list(set(df["Home"]).union(set(df["Away"])))
team_index = {team: i for i, team in enumerate(teams)}
N = len(teams)

# Initialize Massey matrix and score vector
M = np.zeros((N, N))
b = np.zeros(N)

# Fill the matrix and score vector
for _, row in df.iterrows():
    h, a = team_index[row["Home"]], team_index[row["Away"]]
    home_margin = row["Home_Score"] - row["Away_Score"]

    M[h, h] += 1
    M[a, a] += 1
    M[h, a] -= 1
    M[a, h] -= 1

    b[h] += home_margin
    b[a] -= home_margin

# Replace last row to enforce sum constraint (makes matrix invertible)
M[-1, :] = 1
b[-1] = 0

In [ ]:
ratings = np.linalg.solve(M, b)

# Convert ratings to a dictionary
massey_ratings = {team: rating for team, rating in zip(teams, ratings)}

# Sort and display rankings
massey_rankings = sorted(massey_ratings.items(), key=lambda x: x[1], reverse=True)

for rank, (team, rating) in enumerate(massey_rankings, 1):
    print(f"{rank}. {team}: {rating:.2f}")

# Sample Code

In [12]:
# Example dataset
df = pd.DataFrame({
    "Home": ["Kansas", "Arkansas", "Duke"],
    "Home_Score": [78, 70, 65],
    "Away": ["Duke", "Kansas", "Arkansas"],
    "Away_Score": [75, 68, 52]
})

In [13]:
# Get unique teams and index them
teams = list(set(df["Home"]).union(set(df["Away"])))
team_index = {team: i for i, team in enumerate(teams)}
N = len(teams)

# Initialize Massey matrix and score vector
M = np.zeros((N, N))
b = np.zeros(N)

# Fill the matrix and score vector
for _, row in df.iterrows():
    h, a = team_index[row["Home"]], team_index[row["Away"]]
    home_margin = row["Home_Score"] - row["Away_Score"]

    M[h, h] += 1
    M[a, a] += 1
    M[h, a] -= 1
    M[a, h] -= 1

    b[h] += home_margin
    b[a] -= home_margin

# Replace last row to enforce sum constraint (makes matrix invertible)
M[-1, :] = 1
b[-1] = 0

In [14]:
ratings = np.linalg.solve(M, b)

# Convert ratings to a dictionary
massey_ratings = {team: rating for team, rating in zip(teams, ratings)}

# Sort and display rankings
massey_rankings = sorted(massey_ratings.items(), key=lambda x: x[1], reverse=True)

for rank, (team, rating) in enumerate(massey_rankings, 1):
    print(f"{rank}. {team}: {rating:.2f}")

1. Duke: 3.33
2. Kansas: 0.33
3. Arkansas: -3.67
